In [7]:
import pandas as pd
import numpy as np
import sklearn



In [28]:
data_essays = pd.read_csv('data/train_essays.csv')
data_essays_test = pd.read_csv('data/test_essays.csv')


In [26]:
data_essays

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [24]:
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [30]:
# Предполагается, что у вас есть данные, содержащие тексты и соответствующих им авторов.
# Предполагается, что вы уже предобработали тексты и собрали их в формате списка.

# Пример данных
texts = data_essays['text'].tolist()
authors = data_essays['generated'].tolist()

# # Используйте LabelEncoder для преобразования меток авторов в числа
# label_encoder = LabelEncoder()
# encoded_authors = label_encoder.fit_transform(authors)

# Обучите Word2Vec модель на ваших текстах (здесь приведен пример)
# Мы предполагаем, что тексты были предварительно обработаны (токенизированы, удалены стоп-слова и т.д.)
word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)

# Получите векторы для каждого текста, усредняя векторы его слов
def get_text_vector(text):
    vectors = [word2vec_model.wv[word] for word in text.split() if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Преобразуйте все тексты в векторы
text_vectors = [get_text_vector(text) for text in texts]

# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(text_vectors, encoded_authors, test_size=0.2, random_state=42)

# Создайте модель нейронной сети
model = Sequential()
model.add(Dense(64, input_dim=word2vec_model.vector_size, activation='relu'))
model.add(Dense(len(np.unique(authors)), activation='softmax'))

# Скомпилируйте модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучите модель
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32, validation_split=0.2)

# Оцените модель
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
28/28 [==============================] - 1s 23ms/step - loss: 0.3036 - accuracy: 0.9467 - val_loss: 0.0583 - val_accuracy: 1.0000
Epoch 2/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0335 - accuracy: 0.9977 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 3/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0176 - accuracy: 0.9977 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 4/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0165 - accuracy: 0.9977 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 5/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0162 - accuracy: 0.9977 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 6/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0160 - accuracy: 0.9977 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 7/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0160 - accuracy: 0.9977 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 8/10
28/28 [=

KeyError: 'generated'